In [19]:
import os
import numpy as np
import pandas as pd

# Setup

In [20]:
CURR_DIR = os.getcwd()
CURR_DIR

'/Users/willmcgrath/Documents/GitHub/math_693A_class/MATH-693A-Advanced-Numerical-Methods/python'

# Problem 1

### Program the steepest descent and Newton algorithms using the backtracking line search. Use them to minimize the Rosenbrock function $f(x,y)=10(y-x^2)^2 + (1-x)^2$. 
### Initial step length $\alpha_{0} = 1$, $\overline{\alpha} = 1$, $\rho = \frac{1}{2}$, $c = 10^{-4}$. Use initial point $\overline{x}_{0}^{T} = [1.2, 1.2]$ and then the more difficult point $\overline{x}_{0}^{T} = [-1.2, 1]$. 

### The gradient and hessian of the Rosenbrock function is: 
### $\nabla f = \left[\begin{array}{c} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{array}\right] = \left[\begin{array}{c} 40x^3 - 40xy +2x - 2 \\ 20(y-x^2) \end{array}\right]$,  &ensp; $\nabla^{2} f = \left[\begin{array}{c} \frac{\partial f_{x}}{\partial x} \frac{\partial f_{x}}{\partial y} \\ \frac{\partial f_{y}}{\partial x} \frac{\partial f_{y}}{\partial y} \end{array}\right] = \left[\begin{array}{c} 120x^2 - 40y + 2 \ -40x  \\ -40x \ 20 \end{array}\right]$

### The only minimum is at $(x,y)=(1,1)$ where $f(1,1)=0$.

### a) Stop when: $|f(\vec{x}_{k})| < 10^{-8}$. Print first 6 and last 6 values for  for steepest descent and Newton algorithms. Then determine the minimizer of the Rosenbrock function x*.

In [27]:
def test_obj_func(xbar_k):
    x1 = xbar_k[0]
    x2 = xbar_k[1]
    return (x1 + x2**2)**2

def test_gradient(xbar_k):
    x1 = xbar_k[0]
    x2 = xbar_k[1]
    return np.array([2*(x1 + x2**2), 4*x2*(x1 + x2**2)])

xbar_0_trans  = [1, 1]
pbar_0 = - (test_gradient(xbar_0_trans)) / (np.linalg.norm(test_gradient(xbar_0_trans)))
alpha_0 = 1
rho = 1/2
c = 10**-4

new_alpha = alpha_0
xbar_1 = xbar_0_trans + alpha_0*pbar_0 

#first_input = xbar_0_trans.T
#while test_obj_func()

print(pbar_0)
print(xbar_1)

[-0.4472136  -0.89442719]
[0.5527864  0.10557281]


In [21]:
# need to add space in matrices above
# functions 
def objective_func(xbar_k):
    x = xbar_k[0]
    y = xbar_k[1]

    return 10*(y - x**2)**2 + (1 - x)**2

def gradient(xbar_k):
    x = xbar_k[0]
    y = xbar_k[1]

    return np.array([40*x**3 - 40*x*y + 2*x - 2, 20*(y - x**2)])

def hessian(xbar_k):
    x = xbar_k[0]
    y = xbar_k[1]
    
    return np.array([[120*x*x - 40*y + 2, -40*x],[-40*x, 20]])

In [22]:
a = np.array([4, 8])
l1 = np.linalg.norm(a)
l1

8.94427190999916

In [23]:
4*np.sqrt(5)

8.94427190999916

In [ ]:
# steepest descent algorithm (gradient descent using backtracking line search)
# linear convergence

def steepest_descent():
    # use the initial step xbar_0_trans = [1.2, 1.2] 
    xbar_0_trans  = [1.2, 1.2]
    #xbar_k = 
    #while

    # find descent direction pbar_k
    pbar_k = - (gradient(xbar_0_trans)) / (np.linalg.norm(gradient(xbar_0_trans)))

    # set alpha_0 = 1, rho = 1/2, c = 10^-4, and set new_aplha = alpha_0
    alpha_0 = 1
    rho = 1/2
    c = 10**-4
    new_alpha = alpha_0

    # while f(xbar_k + alpha * pbar_k) > f(xbar_k) + c * new_alpha * pbar_k_trans * f_grad(xbar_k)
    input_val = xbar_0_trans + new_alpha * pbar_k
    while 

    # set alpha_k = new_alpha